In [1]:
from threading import Thread
from time import sleep
import socket
import pickle
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Signature.pkcs1_15 import PKCS115_SigScheme
from Crypto.Cipher import PKCS1_v1_5 as Cipher_PKCS1_v1_5
from Crypto.Hash import SHA256
from pprint import pprint

import os
import hashlib

HOST = "192.168.8.5"  # Standard loopback interface address (localhost)
PORT = 9999  # Port to listen on (non-privileged ports are > 1023)

In [2]:
def generate_RSA_key():
    keys = RSA.generate(2048)
    pub  = RSA.import_key(keys.public_key().export_key('DER'))
    prv  = RSA.import_key(keys.export_key('DER'))
    return prv,pub

def load_RSA_key():
    if(os.path.exists("userkey.pickle")):
        try:
            f=open("userkey.pickle","rb")
            prv,pub = pickle.load(f)
            prv = RSA.import_key(prv)
            pub = RSA.import_key(pub)
            return prv,pub
        except Exception as e:
            print(e)
            return None
    else:
        return None

def save_RSA_key(prv,pub):
    keyfile = open("userkey.pickle","wb")
    pickle.dump([prv.export_key(),pub.export_key()],keyfile)
    keyfile.close()
    print("Write success")
    return True

def encrypt(key, data):
    enc_data = Cipher_PKCS1_v1_5.new(key).encrypt(data)
    return enc_data

def decrypt(key,enc_data):
    '''
    Params
        key - Key for decryption of message
        enc - Encrypted message
    '''
    data = Cipher_PKCS1_v1_5.new(key).decrypt(enc_data,None)
    return data

def sign(keypair,data):
    digest = SHA256.new(data)
    signer = PKCS115_SigScheme(keypair)
    signature = signer.sign(digest)
    return signature


def verify(enc_message,signature,publickey):
    '''
    Params
        enc_message - Encrypted Message to be verified.
        signature - Signature of the message.
        publickey - Public of the Private key used to create signature
    '''
    data,enc_og,pubkey = enc_message,signature,publickey
    digest = SHA256.new(data)
    verifier = PKCS115_SigScheme(publickey)
    try:
        verifier.verify(digest, signature)
        return True
    except:
        print("Verify Fail")
        return True

def insertDB(dec_message,public_key,torurl):
    global db
    if(torurl not in db):
        db[torurl]={"messages":[],"pubkey":public_key}
    db[torurl]["messages"].append(dec_message)
    
def process_message(data):
    # Format [torurl,enc_message,signature,public_key]
    data = pickle.loads(data)
    print(data)
    torurl,enc_message,signature,public_key = data
    public_key = RSA.import_key(public_key)
    if(verify(enc_message,signature,public_key)):
        dec_message = decrypt(prv,enc_message)
        print(dec_message)
        insertDB(dec_message,public_key,torurl)
        print("Done!")
        return True
    else:
        print("Verification Failure")
        return False

# Python Server
def receiver_function():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.bind(("0.0.0.0", PORT))
        s.listen()
        while True:
            conn, addr = s.accept()
            with conn:
                # TODO: Threading might give better performance
                while True:
                    # Assuming 1024 bytes as max message size
                    data = conn.recv(1024)
                    if not data:
                        break
                    process_message(data)
    except KeyboardInterrupt:
        print("Bye..")
    
# Client code for the AnonChat server
def send_message(pub,torurl,message):
    remote_host = torurl # URL for reciver 
    PORT = 9999   # Port for AnonChat on reciever 
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((remote_host, PORT))
        enc_message = encrypt(pub,message.encode())
        signature = sign(prv,enc_message)
        s.send(pickle.dumps([HOST,enc_message,signature,pub.export_key()]))
        s.close()

In [3]:
db = {}
keys = load_RSA_key()
prv,pub = (None,None)
if(keys==None):
    print("Keys not avaiable, generating")
    prv,pub = generate_RSA_key()
    print("Saving keys")
    save_RSA_key(prv,pub)
else:
    prv,pub = keys
thread_rec = Thread(target=receiver_function)

In [4]:
thread_rec.start() # Start main reciver thread

In [5]:
def UI_Main():
    chosen_person = None
    while(True):
        try:
            inp = input(">").strip()
            if(inp == "$help"):
                print("Help Menu - Conrgatz you have discovered help menu")
                print("1) $menu for menu")
                print("2) $help for help")
                print("Type a message to chat")
            elif(inp == "$menu"):
                print("1) Contact list")
                print("2) Messages")
                print("3) New Contact")
                print("4) Export Public Key")
                print("5) Print DB")
                print("6) Exit")
                inp = input("Choice?[1]:").strip()
                if(inp == "1"):
                    pprint(db.keys())
                    inp = int(input(f"Choice?[{list(db.keys())[0]}]:").strip())
                    chosen_person = list(db.keys())[inp]
                elif(inp == "2"):
                    if(chosen_person):
                        pprint(db[chosen_person]["messages"])
                    else:
                        print("No selected contact, please select one.")
                elif(inp=="3"):
                    contact = input("Contact url:")
                    db[contact]={"messages":[],"pubkey":None}
                    inp = input("PublicKey:").strip('\' ').encode()
                    key = None
                    try:
                        key = RSA.import_key(inp)
                    except:
                        print("Import Failure! Please try again!")
                    db[contact]["pubkey"] = key
                elif(inp=="4"):
                    print(pub.export_key("OpenSSH").decode())
                elif(inp=="5"):
                    print(db)
                elif(inp=="6"):
                    break;
                else:
                    print("Try again!")
            else:
                if(chosen_person):
                    send_message(db[chosen_person]["pubkey"],chosen_person,inp)
                else:
                    print("No selected contact, please select one.")
        except KeyboardInterrupt:
            print("Exiting!")
            break
        except Exception as e:
            print(e)
            continue

In [ ]:
UI_Main()

>$menu
1) Contact list
2) Messages
3) New Contact
4) Export Public Key
5) Print DB
6) Exit
Choice?[1]:4
ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCbd5wIZBn5cnz1mn74WlKo8s0NkvDzLVnFXBaLqlBy4vBx7s6OVD7MGcsTKvg0E7pSoL5A5Eq2BlxgVOse0tlI+dJKQ/uuiyJ7zw3aP08wHs+94oxyMfeS07k+Pf0MLOSn2R1x2lLGdeR3zoLF6KkPDbfTE89ZUaYitSzDAloFB86GnDhbcMwfRcKUHQK0e1HQ1Pz3Xz54+hW1k3KLaEuwyaT8nV6ocAt8SdjmDSLN4jih4QB4hPiFyFLhH4K1wlEls5b8PDVeADNNFUq1KARGPB+gMSmeqC3lIU9Q7C+XLPjkEpLz4jvONg+QVwvxRHhi8+H3n0N+J4xWEYyoeti9
>$menu
1) Contact list
2) Messages
3) New Contact
4) Export Public Key
5) Print DB
6) Exit
Choice?[1]:3
Contact url:192.168.8.5
PublicKey:ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCbd5wIZBn5cnz1mn74WlKo8s0NkvDzLVnFXBaLqlBy4vBx7s6OVD7MGcsTKvg0E7pSoL5A5Eq2BlxgVOse0tlI+dJKQ/uuiyJ7zw3aP08wHs+94oxyMfeS07k+Pf0MLOSn2R1x2lLGdeR3zoLF6KkPDbfTE89ZUaYitSzDAloFB86GnDhbcMwfRcKUHQK0e1HQ1Pz3Xz54+hW1k3KLaEuwyaT8nV6ocAt8SdjmDSLN4jih4QB4hPiFyFLhH4K1wlEls5b8PDVeADNNFUq1KARGPB+gMSmeqC3lIU9Q7C+XLPjkEpLz4jvONg+QVwvxRHhi8+H3n0N+J4xWEYyoe

In [7]:
signed = b'U\xed\xa8^\xbdm\x13B\x1a\x1cO\xd7\xe7i\xeeG%]\xfa\xcfS\xc6>)\xa5\xc6E\xbch=\xf7Ho\x0e\xd0"\xa9{\xd9\xcf~\xda\xee\x12\x90\x18L\'\xb6\xa2%\xd7\xfc}\\K\xe8\xc8"^\xaa\xd9.T"\xf67\xfc\xd2Q\x86_\x0e]_\x14\x8fHM\x1b"\r\x0b:\xe1\xda]\xb7_#}\xa6\xc7>\xcdc\x0b\xa0\x97&\xb2s\x9b\\\xbc\x9a\xfb\x93\x89Q\xdd\x1c\x01\xaaX\x7f\xb3\xd9\x04y\xb1\xf1D\xd0\xe6\xab\x19\x9a/B\x12l\xae\x18\x95`6\x7fD)\xc6\xd0]*9[\xd08\xf3Ni\xc6\xaeu\x9d\xb4\xa7\xa0\xf4\xb7\x05\xdeq\xebP\x92\x9dE\xbb6\x8a\xbd#\x93"\xe2\xa9\xa2\x00\xe1\xdd6*\x06\x98\xee\xe1C\xdc\xf1\x8c\x07\xba\x95M\xf7(\xafJ\xb3\x0e\xe2\x0e\xcc\x16-\xf6\xd1\xe4\x14i\xe7\x86`PQ\x9eF\xb2\xf9\xa7\x92j\x95\x1cOrX\x7f[\x99\x02/ \xfe\x95\xf2\x91\x01J\xaf|A\x9c\x85\xc6p\xa9\x89\xd0e\x9f\x96\x90\xa2\x06'

In [8]:
key = b'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQDFZNe7E3ruW8//ESTvPDbRoSlZfFU8MDmnhKgA8pufxSyMTm54ApvxK0PHez2h3/cQl1w74neIGUBHko+XodQM00KJc+ZitNzmvY2uHGnHRD4a38huhnq6bG1w+2i6ZvY2Kh08+nWZUb00Psd9yVsyb5mY2AkAwYeIiM0cnzQ5s+KfEoLZpq78yQwvIrbJdL0p6Y/iN2HO9D6O2Sf/z9hTnMdgPAJvOF9kDc3qHCW0YHMx39SUNdjcMLENaM1FMkCvg2HZu348+ISexdvjr9PxZRtj+Do/ZNtWhdHHJTwzTQIoDYICmj56I/ou7GsBjdbjscqE/MvqbpJ+DRvN3Rnh'

In [9]:
key = RSA.import_key(key)

In [11]:
verify("Helloh".encode(),signed,key)

Verify Fail


True

In [12]:
testpriv = RSA.import_key(b'ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABAQCEGkPkgRSkw1wQF4vFns32RZhi0d74HFSvjtHITxOdbbImAbDJIh+2tCzYwHDvXElPiif4McWGbbDjWsZwXuDXs5gZj4KV4wAvD/CM58Jt1l/h2Phqvq7ZF0pe2aG5wJ0E0liqnrO/69K2ZnCl1HalBpzOKFO4ler20PicimJtwqkSKr7IEXj1gPGMQHGVjiPCgRmA/lwSQpThj37SJvq+H1dSAVAUBVt8FuIR7OMk01hDQ2NZmeEDdhYK+SwqWCrAr+7oTg3H1kRL+e61dNBdI7aEZ7eUSAKoTTPIouhmJ63np80TrnOOd4C4NjvgBMOUZYoL5yYZbgsvw3BTB1wT')

In [14]:
testpriv

RsaKey(n=16676408111048271099874660429277627088265003749528702029823942851552414471572529063562948816896009244775008501624760410190049399466117546906924923085322343816820082882532673046691871214514615338541320782079762854897311623268395602528790453402028874812777776543556173720790794739259111034877870106741694567866887491774868365090462706958673983346250196656322265509807746733792786940163567270022161914977066526354599994803697679532989846239150421968205581362072249392305384653949130989986576679715917155526176883671255701248539403773410402737548500519751251463807820879444669917550628592218636113783278057087836373867539, e=65537)

In [15]:
decrypt(testpriv,b'gH*X\xbbT,\xd1r2i5\x88d\xaf\xb2\xe5\xed\xfc\tR\x1e\x83\xf5\xbf\xd5v"\xa5\x9bF\xc8\xf7l82G\x8aU\x84Q\xc8\x89\x0f\xc7\rR\x19\x05\n,Nxs\x13\xbe Z|\xb8\xcf\xdb/2 \xfe\xdft\xa3A\x94\xf8\xee\xad1)G\xab\x11\xda\x03UZ\xdah\xc8\xc7\xd3\xc8{\xc0B\xda\x8e\xdd\xcc\xd2B\xfe\x8a\xd9J\x96Z\xd6\xdd\xa5~\xe6ie\xf8\x86\x17\x81\x14}rc&\xba\xd0\xe7\x1d:\x17h\x92\x1ds\xcfx\xeaP\xd8\x03\xac)\xaf\x9a\xd17\xafb\xf9\xeeK\xb6\x05d\xa1\x98\xe9\x8d%O8\xacFe\x91r^\xcf\xb9 s\x91\x8f\xa6\xb0;\xc2\x1b\xfb|8o@\x9a[D&oa\x8b)\x91\x8b\x1a\xect\xe46\x86\xa0P\x8bQ_\x10\xe6U\xc5{Y\xedx)\xb2\x9d6\n\x82>4\xe8i\xe6\xbc8v\x16\xdaE\x06(\x86\x0cR\xc9\x14\xe1\xc9}F\x03x\x06`\t\xde\x88\xd8^\x8ay\x1f.\x1d\xaf3\\\xb2z-')

TypeError: This is not a private key

In [ ]:
def client_tester():
    HOST = "localhost"
    PORT = 9999
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.connect((HOST, PORT))
        keys = generate_RSA_key() # Priv,Pub
        message = "Hello MOON!"
        enc_message = encrypt(pub,message.encode())
        signature = sign(keys[0],enc_message)
        s.send(pickle.dumps(["localhost",enc_message,signature,keys[1].export_key()]))
        s.close()

In [ ]:
# client_tester()

In [ ]:
db